In [9]:
import import_ipynb
import configparser
import numpy as np
import sys
import time

#--- user modules
sys.path.append(confParser['input files']['lib_path'])
import LammpsPostProcess as lp
import utility as utl
import buildDescriptors as bd
import imp
imp.reload(utl)
imp.reload(lp)
imp.reload(bd)


importing Jupyter notebook from buildDescriptors.ipynb
conf. file sections: ['flags', 'input files', 'EnergyBarrier', 'neural net', 'ml mc']


<module 'buildDescriptors' from 'buildDescriptors.ipynb'>

# MachineLeranedMC

In [14]:
class MachineLeranedMC( bd.ParseConfiguration,
                        bd.EnergyBarrier,
                      ):
    '''
    Performs Machine Learned Monte Carlo Swaps
    '''
    
    def __init__(self,
                 confParser, 
                 verbose = False
                ):
        
#         bd.ParseConfiguration.__init__(self, confParser, verbose = verbose )
        self.verbose     =  verbose 
        self.confParser  =  confParser
        
    def Parse(self,fp):
        t0 = time.time()
        self.lmpData = lp.ReadDumpFile( '%s'%(fp) ) 
        self.lmpData.GetCords( ncount = sys.maxsize)
        if self.verbose:
            print('elapsed time=%s s'%(time.time()-t0))
            print('time steps:',self.lmpData.coord_atoms_broken.keys())
            display(self.lmpData.coord_atoms_broken[0].head())

                    
    def GetDescriptors( self ):
        '''
        Compute structural descriptors 
        '''
        
        bd.EnergyBarrier.__init__( self,
                                  None,#'%s/EVENTS_DIR'%self.confParser['input files']['input_path'],
                                  None,#'%s/EVLIST_DIR'%self.confParser['input files']['input_path'],
                                  self.lmpData,
                                  None,# self.lmpDisp,
                                   verbose    = self.verbose,
                                   nconf      = 2, #--- only two events
                                   confParser = self.confParser,
                                   species    = ['Ni'], #'Ni Co Cr'.split()
                                   r_cut      = eval(self.confParser['EnergyBarrier']['r_cut']),
                                   dr         = eval(self.confParser['EnergyBarrier']['dr']),
                                   scale      = eval(self.confParser['EnergyBarrier']['scale']),
                                   n_max      = 8,
                                   l_max      = 6,

                      )
        
        self.perAtomData = self.lmpData
        self.SetDescriptors(
                      #soap = False,
                      #acsf = True,   
                      gr = True,
                     )
        
    def GetDefects( self ):
        '''
        Classify Defects
        '''
        
        #--- load ml model
        model = keras.models.load_model(fp) #"best_model/convnetClassifier_from_scratch.tf")

        #---------------
        #--- zscore X
        #---------------        
        X      = np.c_[self.descriptors ]
        scaler = StandardScaler()
        scaler.fit(X)
        X      = scaler.transform( X )

        #--- predict classes
        predict_x = model.predict( X ) 
        self.predict_classes = np.argmax( predict_x, axis=1 )
    
    
    def GetDisp( self ):
        '''
        Predict Displacements
        '''

        #--- load ml model
        model = keras.models.load_model(fp) #"best_model/convnetClassifier_from_scratch.tf")
        
        #---------------
        #--- zscore X
        #---------------        
        filtr  = self.predict_classes == 1
        X      = np.c_[self.descriptors[ filtr ] ]
        scaler = StandardScaler()
        scaler.fit(X)
        X      = scaler.transform( X )

        self.predict_disp = model.predict( X )
        
    
    def BuildCatalog( self ):
#         rate_constant_prefactor = ?
        columns = 'AtomId barrier true_rate dx dy dz'.split()


## main()

In [15]:
#--- parse conf. file
confParser = configparser.ConfigParser()
confParser.read('configuration.ini')
print('conf. file sections:',confParser.sections())



mlmc  =  MachineLeranedMC(confParser,verbose = True)
    
#--- parse atom positions
mlmc.Parse('%s/%s'%(confParser['ml mc']['input_path'],confParser['ml mc']['dump_file']))
    
#--- total displacements
# !mkdir disp
# mlmc.Displacement( 'dumpFile/dump.xyz',
#                     'disp/disp.xyz'
#                  )

#--- build descriptors
mlmc.GetDescriptors()

#--- identify defects
mlmc.GetDefects()

#--- predict diffusion paths
mlmc.GetDisp()


#--- build catalog
mlmc.BuildCatalog()

conf. file sections: ['flags', 'input files', 'EnergyBarrier', 'neural net', 'ml mc']
elapsed time=0.010485172271728516 s
time steps: dict_keys([0, 1, 2])


,id,type,x,y,z
0,1,1,-1.600000e-07,-3.400000e-07,-3.497377
1,2,1,1.964940e-02,3.538844e+00,0.019379
2,3,1,2.030230e-02,1.176100e-04,3.561652
3,4,1,1.898460e-02,-3.538610e+00,0.019878
4,5,1,-3.498158e+00,1.170700e-04,-0.000241


Build neighbor list: elapsed time=2.301715850830078 s


,id,type,J,Jtype,DIST,DX,DY,DZ,PBC_SHIFT_X,PBC_SHIFT_Y,PBC_SHIFT_Z
0,1,1,7.0,1.0,2.472952,-0.000685,1.748593,1.748690,0.0,1.0,0.0
1,1,1,11.0,1.0,2.473517,0.000212,-1.749392,1.748690,1.0,0.0,0.0
2,1,1,8.0,1.0,2.504166,1.792465,0.000412,1.748690,1.0,1.0,0.0
3,2,1,9.0,1.0,2.516353,-1.768929,-1.789567,-0.019379,-1.0,0.0,-1.0
4,2,1,7.0,1.0,2.516241,-0.020334,-1.790251,-1.768066,-1.0,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...
101,18,1,7.0,1.0,2.472951,-0.000685,1.748593,-1.748688,-1.0,0.0,-1.0
102,18,1,16.0,1.0,2.504214,-1.749029,0.000249,1.792200,-1.0,0.0,0.0
103,18,1,11.0,1.0,2.473516,0.000212,-1.749392,-1.748688,0.0,-1.0,-1.0
104,18,1,15.0,1.0,2.515842,1.789368,-1.768402,0.019474,0.0,-1.0,0.0


xdim= 21
ydim= 21
zdim= 21


LinAlgError: singular matrix

In [ ]:
mlmc.descriptors

In [ ]:
dir(mlmc)